In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.facecolor'] = 'grey'


X = X_C
y = y
features = df2.columns.values.tolist()

random_seed = 42
cv_rounds = 10
rf = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=random_seed)

cv_folds = StratifiedKFold(n_splits=cv_rounds, shuffle=True, random_state=random_seed)


scores = {}
for feat in features:
    scores[feat] = []
d = 0
for train_idx, test_idx in cv_folds.split(X,y):
    d += 1
    print "Round %d" % d
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = y[train_idx], y[test_idx]
    r = rf.fit(X_train, Y_train)
    acc = accuracy_score(Y_test, rf.predict(X_test))
    for i in tqdm.tqdm_notebook(range(X.shape[1])):
        X_t = X_test.copy()
        np.random.shuffle(X_t[:, i])
        shuff_acc = accuracy_score(Y_test, rf.predict(X_t))
        scores[features[i]].append((acc-shuff_acc)/acc)

mda_features = df.columns.values.tolist()
mda_importance = [(np.mean(score)) for score in scores.values()]
mda_indices = np.argsort(mda_importance)

plt.title('Feature Importances')
plt.barh(range(len(mda_indices)), [mda_importance[i] for i in mda_indices], color='#8f63f4', align='center')
plt.yticks(range(len(mda_indices)), [mda_features[i] for i in mda_indices])
plt.xlabel('Mean decrease accuracy')
plt.show()